### 개선판 word2vec 학습

### CBOW 모델 구현

In [5]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


In [6]:
# coding: utf-8
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.layers import *
from ch04.negative_sampling_layer import NegativeSamplingLoss


class SkipGram:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        W_in = 0.01 * rn(V, H).astype('f')
        W_out = 0.01 * rn(V, H).astype('f')

        # 계층 생성
        self.in_layer = Embedding(W_in)
        self.loss_layers = []
        for i in range(2 * window_size):
            layer = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
            self.loss_layers.append(layer)

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer] + self.loss_layers
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = self.in_layer.forward(target)

        loss = 0
        for i, layer in enumerate(self.loss_layers):
            loss += layer.forward(h, contexts[:, i])
        return loss

    def backward(self, dout=1):
        dh = 0
        for i, layer in enumerate(self.loss_layers):
            dh += layer.backward(dout)
        self.in_layer.backward(dh)
        return None


### CBOW 모델 학습 코드

In [7]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 2[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 5[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 8[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 10[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 13[s] | 손실 3.92
| 에폭 1 |  반복 121 / 9295 | 시간 16[s] | 손실 3.79
| 에폭 1 |  반복 141 / 9295 | 시간 18[s] | 손실 3.63
| 에폭 1 |  반복 161 / 9295 | 시간 20[s] | 손실 3.49
| 에폭 1 |  반복 181 / 9295 | 시간 22[s] | 손실 3.36
| 에폭 1 |  반복 201 / 9295 | 시간 24[s] | 손실 3.25
| 에폭 1 |  반복 221 / 9295 | 시간 26[s] | 손실 3.18
| 에폭 1 |  반복 241 / 9295 | 시간 27[s] | 손실 3.07
| 에폭 1 |  반복 261 / 9295 | 시간 29[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 31[s] | 손실 2.96
| 에폭 1 |  반복 301 / 9295 | 시간 33[s] | 손실 2.91
| 에폭 1 |  반복 321 / 9295 | 시간 35[s] | 손실 2.88
| 에폭 1 |  반복 341 / 9295 | 시간 37[s] | 손실 2.86
| 에폭 1 |  반복 361 / 9295 | 시간 39[s] | 손실 2.80
| 에폭 1 |  반복 381 / 9295 | 시간 41[s] | 손실 2.78
| 에폭 1 |  반복 401 / 9295 | 시간 43[s] | 손실 2.77
| 에폭 1 |  반복 421 / 9295 | 시간 45[s] | 손실 2.74
| 에폭 1 |  반복 441 / 9

| 에폭 1 |  반복 3541 / 9295 | 시간 369[s] | 손실 2.44
| 에폭 1 |  반복 3561 / 9295 | 시간 371[s] | 손실 2.41
| 에폭 1 |  반복 3581 / 9295 | 시간 373[s] | 손실 2.42
| 에폭 1 |  반복 3601 / 9295 | 시간 375[s] | 손실 2.42
| 에폭 1 |  반복 3621 / 9295 | 시간 377[s] | 손실 2.42
| 에폭 1 |  반복 3641 / 9295 | 시간 379[s] | 손실 2.40
| 에폭 1 |  반복 3661 / 9295 | 시간 381[s] | 손실 2.39
| 에폭 1 |  반복 3681 / 9295 | 시간 383[s] | 손실 2.39
| 에폭 1 |  반복 3701 / 9295 | 시간 385[s] | 손실 2.44
| 에폭 1 |  반복 3721 / 9295 | 시간 387[s] | 손실 2.42
| 에폭 1 |  반복 3741 / 9295 | 시간 389[s] | 손실 2.41
| 에폭 1 |  반복 3761 / 9295 | 시간 392[s] | 손실 2.43
| 에폭 1 |  반복 3781 / 9295 | 시간 394[s] | 손실 2.42
| 에폭 1 |  반복 3801 / 9295 | 시간 396[s] | 손실 2.39
| 에폭 1 |  반복 3821 / 9295 | 시간 398[s] | 손실 2.39
| 에폭 1 |  반복 3841 / 9295 | 시간 400[s] | 손실 2.40
| 에폭 1 |  반복 3861 / 9295 | 시간 403[s] | 손실 2.41
| 에폭 1 |  반복 3881 / 9295 | 시간 405[s] | 손실 2.40
| 에폭 1 |  반복 3901 / 9295 | 시간 408[s] | 손실 2.41
| 에폭 1 |  반복 3921 / 9295 | 시간 410[s] | 손실 2.38
| 에폭 1 |  반복 3941 / 9295 | 시간 412[s] | 손실 2.41
| 에폭 1 |  반복 

| 에폭 1 |  반복 7041 / 9295 | 시간 729[s] | 손실 2.25
| 에폭 1 |  반복 7061 / 9295 | 시간 731[s] | 손실 2.27
| 에폭 1 |  반복 7081 / 9295 | 시간 734[s] | 손실 2.24
| 에폭 1 |  반복 7101 / 9295 | 시간 736[s] | 손실 2.28
| 에폭 1 |  반복 7121 / 9295 | 시간 738[s] | 손실 2.28
| 에폭 1 |  반복 7141 / 9295 | 시간 740[s] | 손실 2.29
| 에폭 1 |  반복 7161 / 9295 | 시간 742[s] | 손실 2.27
| 에폭 1 |  반복 7181 / 9295 | 시간 744[s] | 손실 2.28
| 에폭 1 |  반복 7201 / 9295 | 시간 747[s] | 손실 2.27
| 에폭 1 |  반복 7221 / 9295 | 시간 749[s] | 손실 2.28
| 에폭 1 |  반복 7241 / 9295 | 시간 752[s] | 손실 2.28
| 에폭 1 |  반복 7261 / 9295 | 시간 754[s] | 손실 2.28
| 에폭 1 |  반복 7281 / 9295 | 시간 757[s] | 손실 2.30
| 에폭 1 |  반복 7301 / 9295 | 시간 759[s] | 손실 2.30
| 에폭 1 |  반복 7321 / 9295 | 시간 761[s] | 손실 2.28
| 에폭 1 |  반복 7341 / 9295 | 시간 763[s] | 손실 2.27
| 에폭 1 |  반복 7361 / 9295 | 시간 765[s] | 손실 2.26
| 에폭 1 |  반복 7381 / 9295 | 시간 767[s] | 손실 2.26
| 에폭 1 |  반복 7401 / 9295 | 시간 769[s] | 손실 2.29
| 에폭 1 |  반복 7421 / 9295 | 시간 771[s] | 손실 2.24
| 에폭 1 |  반복 7441 / 9295 | 시간 773[s] | 손실 2.28
| 에폭 1 |  반복 

| 에폭 2 |  반복 1241 / 9295 | 시간 1095[s] | 손실 2.09
| 에폭 2 |  반복 1261 / 9295 | 시간 1097[s] | 손실 2.13
| 에폭 2 |  반복 1281 / 9295 | 시간 1099[s] | 손실 2.12
| 에폭 2 |  반복 1301 / 9295 | 시간 1102[s] | 손실 2.13
| 에폭 2 |  반복 1321 / 9295 | 시간 1105[s] | 손실 2.16
| 에폭 2 |  반복 1341 / 9295 | 시간 1108[s] | 손실 2.17
| 에폭 2 |  반복 1361 / 9295 | 시간 1111[s] | 손실 2.15
| 에폭 2 |  반복 1381 / 9295 | 시간 1113[s] | 손실 2.14
| 에폭 2 |  반복 1401 / 9295 | 시간 1116[s] | 손실 2.15
| 에폭 2 |  반복 1421 / 9295 | 시간 1118[s] | 손실 2.14
| 에폭 2 |  반복 1441 / 9295 | 시간 1122[s] | 손실 2.17
| 에폭 2 |  반복 1461 / 9295 | 시간 1124[s] | 손실 2.12
| 에폭 2 |  반복 1481 / 9295 | 시간 1126[s] | 손실 2.16
| 에폭 2 |  반복 1501 / 9295 | 시간 1128[s] | 손실 2.09
| 에폭 2 |  반복 1521 / 9295 | 시간 1131[s] | 손실 2.11
| 에폭 2 |  반복 1541 / 9295 | 시간 1133[s] | 손실 2.14
| 에폭 2 |  반복 1561 / 9295 | 시간 1135[s] | 손실 2.17
| 에폭 2 |  반복 1581 / 9295 | 시간 1137[s] | 손실 2.17
| 에폭 2 |  반복 1601 / 9295 | 시간 1139[s] | 손실 2.18
| 에폭 2 |  반복 1621 / 9295 | 시간 1141[s] | 손실 2.17
| 에폭 2 |  반복 1641 / 9295 | 시간 1143[s] | 

KeyboardInterrupt: 

###  CBOW 모델 평가

In [8]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

# 이전 모델들보다 훨씬 연관있어 보이는 단어들이 출력됨.


[query] you
 we: 0.71533203125
 i: 0.705078125
 your: 0.626953125
 they: 0.6171875
 anything: 0.59716796875

[query] year
 month: 0.83837890625
 week: 0.787109375
 spring: 0.767578125
 summer: 0.759765625
 decade: 0.697265625

[query] car
 truck: 0.6337890625
 luxury: 0.6328125
 auto: 0.5751953125
 window: 0.57421875
 cars: 0.55859375

[query] toyota
 honda: 0.658203125
 marathon: 0.6279296875
 coated: 0.62646484375
 engines: 0.61962890625
 seita: 0.61669921875


In [9]:
# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs) 
# 'king'과 'man'이 관계가 있으면 'queen' 다음은 woman이 나와야함
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
# 'take'과 'took'이 관계가 있으면 'go' 다음은 went이 나와야함
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)

# 잘 유추해내는 것을 알 수 있음

--------------------------------------------------

[analogy] king:man = queen:?
 a.m: 5.7421875
 woman: 5.50390625
 naczelnik: 5.10546875
 carolinas: 4.93359375
 hacker: 4.74609375

[analogy] take:took = go:?
 went: 4.6796875
 're: 4.45703125
 came: 4.4140625
 were: 4.07421875
 was: 4.0625

[analogy] car:cars = child:?
 a.m: 6.75390625
 rape: 5.875
 children: 5.3203125
 daffynition: 4.734375
 women: 4.5390625

[analogy] good:better = bad:?
 rather: 5.78125
 more: 5.73828125
 less: 5.40625
 greater: 4.5078125
 fewer: 4.1328125
